In [1]:
import os
import tifffile

# 경로 설정
folder = "tif"
files = [f for f in os.listdir(folder) if f.endswith('.tif')]

for fname in files:
    path = os.path.join(folder, fname)
    with tifffile.TiffFile(path) as tif:
        print(f"{fname}: {len(tif.pages)} frames")


sample_stack_4.tif: 3923 frames
1_stack.tif: 95 frames
3_stack.tif: 3880 frames
4_stack.tif: 1 frames
6_stack.tif: 3923 frames
DRG_20x_ROI1_1_shapen.tif: 3597 frames
DRG_20x_ROI1_2_shapen.tif: 3597 frames
DRG_20x_ROI1_3_shapen.tif: 3597 frames
DRG_20x_ROI2_1_shapen.tif: 3597 frames
DRG_20x_ROI2_2_shapen.tif: 3597 frames
DRG_20x_ROI2_3_shapen.tif: 3597 frames
DRG_20x_ROI2_4_shapen.tif: 3597 frames
DRG_20x_ROI2_5_shapen.tif: 3597 frames
DRG_20x_ROI2_6_shapen.tif: 3597 frames
DRG_20x_ROI3_1_shapen.tif: 3597 frames
DRG_20x_ROI3_2_shapen.tif: 3597 frames
DRG_20x_ROI3_3_shapen.tif: 3597 frames
DRG_20x_ROI3_4_shapen.tif: 3597 frames
DRG_20x_ROI4_1_shapen.tif: 3597 frames
DRG_20x_ROI4_2_shapen.tif: 3597 frames
DRG_20x_ROI4_3_shapen.tif: 3597 frames
DRG_20x_ROI5_1_shapen.tif: 3597 frames
DRG_20x_ROI5_2_shapen.tif: 3597 frames
DRG_20x_ROI5_3_shapen.tif: 3597 frames
DRG_20x_ROI5_4_shapen.tif: 3597 frames
DRG_20x_ROI5_5_shapen.tif: 3597 frames
DRG_20x_STIM_CAP_300uM_ROI1_1_shapen.tif: 3597 frames


In [4]:
import os
import tifffile
import numpy as np
from PIL import Image
from cellpose import models
import random

# 설정
tif_dir = "E:/DataStorage/tif"  # TIF 폴더 경로
output_img_dir = "E:/DataStorage/UNet_train/img"
output_mask_dir = "E:/DataStorage/UNet_train/mask"
os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_mask_dir, exist_ok=True)

# Cellpose 모델 로드 (GPU 설정 가능)
cellpose_model = models.CellposeModel(gpu=False, model_type='cyto')

# 각 tif에서 5개 프레임 추출
all_tifs = [f for f in os.listdir(tif_dir) if f.endswith(".tif")]
global_count = 0

for tif_name in all_tifs:
    tif_path = os.path.join(tif_dir, tif_name)
    with tifffile.TiffFile(tif_path) as tif:
        n_frames = len(tif.pages)
        if n_frames < 5:
            continue

        indices = sorted(random.sample(range(n_frames), 5))

        for idx in indices:
            img = tif.pages[idx].asarray()

            # ⚠️ 어두운 프레임은 건너뛰기
            if np.mean(img) < 5:
                continue

            # ⚠️ uint8 변환 (0~255 정규화)
            if img.max() > 0:
                img = (img / img.max()) * 255
            img = img.astype(np.uint8)

            # 저장 파일 이름
            img_fname = f"img_{global_count:04d}.png"
            img_path = os.path.join(output_img_dir, img_fname)
            mask_path = os.path.join(output_mask_dir, img_fname.replace(".png", "_mask.png"))

            # 이미지 저장
            img_pil = Image.fromarray(img)
            img_pil.save(img_path)

            # Cellpose 마스크 생성 및 저장
            masks, _, _ = cellpose_model.eval([img], diameter=None, channels=[0, 0])
            mask_img = Image.fromarray(masks[0].astype(np.uint8) * 255)
            mask_img.save(mask_path)

            global_count += 1

print(f"[INFO] 총 {global_count}장의 이미지/마스크가 저장되었습니다.")


model_type argument is not used in v4.0.1+. Ignoring this argument...
channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used
channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used
channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used
channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used
channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used
channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used
channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used
channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used
channels deprecated in v4.0.1+. If data contain more than 

[INFO] 총 165장의 이미지/마스크가 저장되었습니다.
